# GxE problem

- I updated the simulation function so that sigma is always positive (exponential)
- I am using `yt` in the function below to estimate models

In [17]:
library(brms)
library(data.table)
library(texreg)

In [7]:
# Domingue's simulation function (adjusted)
simData = function(E,i=1,b0=.8,b1=.2,b2=0.2,b3=.15,h=sqrt(.6), a0 = 0, a1=.5, sigma=1,scaling=TRUE) {
    N = length(E)
    G = rnorm(N,0,1)
    eps = rnorm(N,1,sigma)
    if (scaling){
        e = sqrt(1-h^2)
        # I don't use ystar and y
        ystar = h*G+e*eps
        y = a0 + a1*E+(b0 + b1*E)*ystar
        # sigma of the error term should be
        fsigma = exp(b0*e + b1*e*E) 
        # final y values
        yt = a0 + a1*E + b0*h*G + b1*h*E*G + rnorm(N, 0, fsigma)

    } else {   
        y = b1*G + b2*E+ b3*G*E + eps
        fsigma = exp(0.1 + 0.4*E) 
        yt = a0 + b1*G + b2*E+ b3*G*E + rnorm(N, 0, fsigma)
    }
    df = data.frame( E=E, y=y,yt = yt, g=G)
    df
}


## Scaling model

In [8]:
E = rnorm(5000, 0, 1)
dts = data.table(simData(E, scaling = TRUE))
summary(dts)



       E                   y                 yt                 g            
 Min.   :-3.422758   Min.   :-2.2119   Min.   :-7.96256   Min.   :-4.147215  
 1st Qu.:-0.679987   1st Qu.:-0.2451   1st Qu.:-1.29524   1st Qu.:-0.649639  
 Median :-0.001713   Median : 0.3913   Median :-0.10122   Median :-0.000141  
 Mean   :-0.000054   Mean   : 0.4979   Mean   :-0.01383   Mean   : 0.005990  
 3rd Qu.: 0.682829   3rd Qu.: 1.1286   3rd Qu.: 1.14081   3rd Qu.: 0.655652  
 Max.   : 3.797076   Max.   : 6.0249   Max.   : 9.93467   Max.   : 3.642731  

In [9]:
cnames = c("ystar", "y")
m1 = lm(y ~ g + E + g * E, data = dts)
m2 = lm(yt ~ g + E + g * E, data = dts)
cat(screenreg(list(m1, m2)))


             Model 1      Model 2    
-------------------------------------
(Intercept)     0.50 ***    -0.01    
               (0.01)       (0.02)   
g               0.62 ***     0.62 ***
               (0.01)       (0.02)   
E               0.64 ***     0.51 ***
               (0.01)       (0.02)   
g:E             0.17 ***     0.22 ***
               (0.01)       (0.02)   
-------------------------------------
R^2             0.75         0.19    
Adj. R^2        0.75         0.19    
Num. obs.    5000         5000       
*** p < 0.001; ** p < 0.01; * p < 0.05



# Bayesian distributional model


## Scaling model

In [ ]:
# distributional model using bayesian stats
f = bf(yt ~ g + E + g * E, sigma ~ 1 + E)
m3 = brm(f, data = dts, family = brmsfamily("gaussian", link_sigma = "log"))


In [11]:
# able to get the sigma coefficients of the simulation
cat(screenreg(m3))


                 Model 1      
------------------------------
Intercept           -0.01     
                 [-0.05; 0.03]
sigma_Intercept      0.51 *   
                 [ 0.49; 0.53]
g                    0.62 *   
                 [ 0.58; 0.66]
E                    0.51 *   
                 [ 0.47; 0.55]
g:E                  0.20 *   
                 [ 0.16; 0.23]
sigma_E              0.13 *   
                 [ 0.11; 0.14]
------------------------------
R^2                  0.19     
Num. obs.         5000        
loo IC           19301.98     
WAIC             19301.96     
* 0 outside the confidence interval.


In [12]:

# I cannot reject the null hypothesis
hyp <- "g * sigma_E = g:E * sigma_Intercept"
(hyp <- hypothesis(m3, hyp, alpha = 0.05))


Hypothesis Tests for class b:
                Hypothesis Estimate Est.Error CI.Lower CI.Upper Evid.Ratio
1 (g*sigma_E)-(g:E*... = 0    -0.02      0.01    -0.05     0.01         NA
  Post.Prob Star
1        NA     
---
'CI': 90%-CI for one-sided and 95%-CI for two-sided hypotheses.
'*': For one-sided hypotheses, the posterior probability exceeds 95%;
for two-sided hypotheses, the value tested against lies outside the 95%-CI.
Posterior probabilities of point hypotheses assume equal prior probabilities.

## Non-scaling model

In [13]:
dt = data.table(simData(E, scaling = FALSE))
summary(dt)

       E                   y                 yt                 g            
 Min.   :-3.422758   Min.   :-2.1670   Min.   :-5.57675   Min.   :-3.816905  
 1st Qu.:-0.679987   1st Qu.: 0.3471   1st Qu.:-0.77644   1st Qu.:-0.680667  
 Median :-0.001713   Median : 1.0098   Median :-0.09029   Median :-0.005071  
 Mean   :-0.000054   Mean   : 1.0248   Mean   : 0.01564   Mean   :-0.009181  
 3rd Qu.: 0.682829   3rd Qu.: 1.6896   3rd Qu.: 0.73278   3rd Qu.: 0.656683  
 Max.   : 3.797076   Max.   : 4.8681   Max.   :12.08294   Max.   : 3.876883  

In [ ]:
f = bf(yt ~ g + E + g * E, sigma ~ 1 + E)
m4 = brm(f, data = dt, family = brmsfamily("gaussian", link_sigma = "log"))

In [15]:
cat(screenreg(m4))


                 Model 1      
------------------------------
Intercept            0.01     
                 [-0.01; 0.04]
sigma_Intercept      0.10 *   
                 [ 0.08; 0.12]
g                    0.19 *   
                 [ 0.16; 0.21]
E                    0.21 *   
                 [ 0.19; 0.24]
g:E                  0.14 *   
                 [ 0.12; 0.16]
sigma_E              0.39 *   
                 [ 0.37; 0.40]
------------------------------
R^2                  0.06     
Num. obs.         5000        
loo IC           15191.64     
WAIC             15191.62     
* 0 outside the confidence interval.


In [16]:
# I reject the null hypothesis
hyp <- "g * sigma_E = g:E * sigma_Intercept"
(hyp <- hypothesis(m4, hyp, alpha = 0.05))

Hypothesis Tests for class b:
                Hypothesis Estimate Est.Error CI.Lower CI.Upper Evid.Ratio
1 (g*sigma_E)-(g:E*... = 0     0.06      0.01     0.05     0.07         NA
  Post.Prob Star
1        NA    *
---
'CI': 90%-CI for one-sided and 95%-CI for two-sided hypotheses.
'*': For one-sided hypotheses, the posterior probability exceeds 95%;
for two-sided hypotheses, the value tested against lies outside the 95%-CI.
Posterior probabilities of point hypotheses assume equal prior probabilities.